# Preprocess clean database

In this new notebook a dataset is created containing only pure folded or disordered regions of the proteins from the PED and SCOPe database. From every protein, one longest folded and longest disordered region is selected. The regions are classified as disordered or folded based on secondary structure, where: 'H' (helix) and 'E' (elongated) are classified as folded, and 'C' (coil) is classified as disordered. The region must be at least 5 residues long and can have maximum three interruptions of the other secondary structure. 

To use this code, first the PED and SCOPe trajectory has to be available: '/Users/murielhammond/RP1/CT_code/CT_analysis/trajectory/5/'. This folder contains all single_frame PDB files of PED and SCOPe dataframe.

## Importing libraries

In [4]:
import numpy as np
import os
import pandas as pd

import ast
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import mdtraj as md
import itertools

import plotly.graph_objects as go
import plotly.express as px

from pathlib import Path
from collections import defaultdict
import shutil

## Functions

In [5]:
# Function to find the longest folded region per protein
def find_folded_region(ss_vector):
    best_start, best_end = None, None
    max_len = 0

    i = 0
    while i < len(ss_vector):
        if ss_vector[i] in ['H', 'E']:
            interrupts = 0
            j = i
            count_HE = 0
            while j < len(ss_vector):
                if ss_vector[j] in ['H', 'E']:
                    count_HE += 1
                elif ss_vector[j] == 'C':
                    interrupts += 1
                    if interrupts > 3:
                        break
                else:
                    break
                j += 1
            if count_HE >= 5 and (j - i) > max_len:
                best_start, best_end = i, j
                max_len = j - i
        i += 1

    return (best_start, best_end)

# Function to find the longest disordered region per protein (so one region for all ensembles of a protein)
def find_disordered_region(ss_vector):
    best_start, best_end = None, None
    max_len = 0

    i = 0
    while i < len(ss_vector):
        if ss_vector[i] == 'C':
            j = i
            while j < len(ss_vector) and ss_vector[j] == 'C':
                j += 1
            region_len = j - i
            if region_len >= 5 and region_len > max_len:
                best_start, best_end = i, j
                max_len = region_len
            i = j
        else:
            i += 1

    return (best_start, best_end)

# Function that takes the region indices of the previous functions and splices the proteins based on these indices
def extract_regions_from_folder(folder_path):
    folder = Path(folder_path)
    pdb_files = list(folder.glob("*.pdb"))

    # Group by protid (e.g. from PROTID_abc.pdb → PROTID)
    protid_groups = defaultdict(list)
    for pdb_file in pdb_files:
        protid = pdb_file.stem.split('_')[0]
        protid_groups[protid].append(pdb_file)

    folded_dict = {}
    disordered_dict = {}

    for protid, files in protid_groups.items():
        best_fol_len = 0
        best_dis_len = 0
        best_fol = (None, None)
        best_dis = (None, None)

        for file in files:
            try:
                traj = md.load(str(file))
                ss_vector = md.compute_dssp(traj, simplified=True)[0].tolist()
            except:
                continue

            fol_region = find_folded_region(ss_vector)
            dis_region = find_disordered_region(ss_vector)

            if fol_region[0] is not None:
                fol_len = fol_region[1] - fol_region[0]
                if fol_len > best_fol_len:
                    best_fol = fol_region
                    best_fol_len = fol_len
                    best_fol_ss = ''.join(ss_vector[fol_region[0]:fol_region[1]])

            if dis_region[0] is not None:
                dis_len = dis_region[1] - dis_region[0]
                if dis_len > best_dis_len:
                    best_dis = dis_region
                    best_dis_len = dis_len
                    best_dis_ss = ''.join(ss_vector[dis_region[0]:dis_region[1]])

        folded_dict[protid] = {'indices': best_fol, 'ss': best_fol_ss}
        disordered_dict[protid] = {'indices': best_dis, 'ss': best_dis_ss}

    folded_data = []
    disordered_data = []
    
    for protid in protid_groups.keys():
        fol_entry = folded_dict.get(protid)
        if fol_entry and fol_entry['indices'] != (None, None):
            folded_data.append({
                'Protid': protid,
                'Region_ss': fol_entry['ss'],
                'Region_indices': fol_entry['indices'],
                'Region': 'Folded'
            })
    
        dis_entry = disordered_dict.get(protid)
        if dis_entry and dis_entry['indices'] != (None, None):
            disordered_data.append({
                'Protid': protid,
                'Region_ss': dis_entry['ss'],
                'Region_indices': dis_entry['indices'],
                'Region': 'Disordered'
            })
        
    folded_df = pd.DataFrame(folded_data)
    disordered_df = pd.DataFrame(disordered_data)
    
    return folded_df, disordered_df

In [7]:
# Running this function extracts every longest disordered and folded region per protein from the trajectory folder (5 contains SCOPe and PED proteins together)
folded_df, disordered_df = extract_regions_from_folder('/Users/murielhammond/RP1/CT_code/CT_analysis/trajectory/5/')

,Protid,Region_ss,Region_indices,Region
0,PED00258,HHHHHHHHHHHHHHHHHHEECCEECHHHHHHHHHHHHH,"(6, 44)",Folded
1,PED00213,EEEEEEEEECCHHHHHHHHHHC,"(53, 75)",Folded
2,PED00366,HHHHHHHHHCCC,"(9, 21)",Folded
3,PED00022,HHHHCCCHHHHHHHHHHHHHHHHHHHH,"(1, 28)",Folded
4,PED00192,EEEECCEEEEEC,"(41, 53)",Folded
...,...,...,...,...
788,1ZD0,EEEEEEEECCCHHHHHHHHHHH,"(95, 117)",Folded
789,1RO2,HHHEEEEEECCHHHHHHHHCHHHHHHHHHH,"(64, 94)",Folded
790,3S4E,HHHHHHHHHHHHHHCCCHHHHHHHHHHH,"(93, 121)",Folded
791,4A2V,EEEEEEEECCEEEEEEECHHHEEE,"(67, 91)",Folded


In [8]:
# folded_df.to_csv('/Users/murielhammond/RP1/CT_code/CT_analysis/Dataframes/df_folded_regions_v0')
# disordered_df.to_csv('/Users/murielhammond/RP1/CT_code/CT_analysis/Dataframes/df_disordered_regions_v0')

In [16]:
folded_df = pd.read_csv('/Users/murielhammond/RP1/CT_code/CT_analysis/Dataframes/df_folded_regions_v0', index_col=0)
disordered_df = pd.read_csv('/Users/murielhammond/RP1/CT_code/CT_analysis/Dataframes/df_disordered_regions_v0', index_col=0)

## Write out indices

In [20]:
# Indinces in the dataframe are written as 1,2,3,4 instead of 1-4.
disordered_df['Region_indices'] = disordered_df['Region_indices'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
folded_df['Region_indices'] = folded_df['Region_indices'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
def expand_tuple_to_range(x):
    if isinstance(x, str):
        x = ast.literal_eval(x)
    if isinstance(x, tuple) and len(x) == 2:
        return list(range(x[0], x[1] + 1))
    return x

# Apply to both dataframes
disordered_df['Region_indices'] = disordered_df['Region_indices'].apply(expand_tuple_to_range)
folded_df['Region_indices'] = folded_df['Region_indices'].apply(expand_tuple_to_range)

## Slice pdb files on indices (for every ensemble)

In [21]:
# Use indices to split the disordered proteins on correct indices and save them
input_folder = '/Users/murielhammond/RP1/CT_code/CT_analysis/trajectory/5/'
output_folder = '/Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged/'
os.makedirs(output_folder, exist_ok=True)

# Make list of all PDB files and iterrate over these files to split them
all_pdb_files = [f for f in os.listdir(input_folder) if f.endswith('.pdb')]

for _, row in disordered_df.iterrows():
    protid = row['Protid']
    region_residue_indices = row['Region_indices']

    matching_files = [f for f in all_pdb_files if f.startswith(protid)]

    for pdb_file in matching_files:
        input_path = os.path.join(input_folder, pdb_file)
        output_path = os.path.join(output_folder, f'{pdb_file[:-4]}_disordered.pdb')

        try:
            traj = md.load(input_path)

            # Convert residue indices to atom indices
            atom_indices = [
                atom.index
                for atom in traj.topology.atoms
                if atom.residue.index in region_residue_indices
            ]
            if not atom_indices:
                print(f"No atom indices found for {pdb_file} with residues {region_residue_indices}")
                continue

            sliced_traj = traj.atom_slice(atom_indices)
            sliced_traj.save_pdb(output_path)
            print(f"Saved: {output_path}")

        except Exception as e:
            print(f"Error processing {pdb_file}: {e}")

Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged/PED00258_e001.25_disordered.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged/PED00258_e001.31_disordered.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged/PED00258_e001.19_disordered.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged/PED00258_e001.18_disordered.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged/PED00258_e001.30_disordered.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged/PED00258_e001.24_disordered.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged/PED00258_e001.32_disordered.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged/

In [22]:
# Use indices to split the folded proteins on correct indices and save them
input_folder = '/Users/murielhammond/RP1/CT_code/CT_analysis/trajectory/5/'
output_folder = '/Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged/'
os.makedirs(output_folder, exist_ok=True)

# Make list of all PDB files and iterrate over these files to split them
all_pdb_files = [f for f in os.listdir(input_folder) if f.endswith('.pdb')]

for _, row in folded_df.iterrows():
    protid = row['Protid']
    region_residue_indices = row['Region_indices']

    matching_files = [f for f in all_pdb_files if f.startswith(protid)]

    for pdb_file in matching_files:
        input_path = os.path.join(input_folder, pdb_file)
        output_path = os.path.join(output_folder, f'{pdb_file[:-4]}_folded.pdb')

        try:
            traj = md.load(input_path)

            # Convert residue indices to atom indices
            atom_indices = [
                atom.index
                for atom in traj.topology.atoms
                if atom.residue.index in region_residue_indices
            ]

            if not atom_indices:
                print(f"No atom indices found for {pdb_file} with residues {region_residue_indices}")
                continue

            sliced_traj = traj.atom_slice(atom_indices)
            sliced_traj.save_pdb(output_path)
            print(f"Saved: {output_path}")

        except Exception as e:
            print(f"Error processing {pdb_file}: {e}")

Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged/PED00258_e001.25_folded.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged/PED00258_e001.31_folded.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged/PED00258_e001.19_folded.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged/PED00258_e001.18_folded.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged/PED00258_e001.30_folded.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged/PED00258_e001.24_folded.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged/PED00258_e001.32_folded.pdb
Saved: /Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged/PED00258_e001.26_folded.pdb
Saved: /Users/murielhammond/RP1/

## Only save lines starting with 'ATOM' and skip hydrogen atoms

In [23]:
# Go through folder and delete all lines from folders that don't start with \"ATOM\"\n",
files = sorted([f for f in os.listdir('/Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged') if os.path.isfile(os.path.join('/Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged'))and not f.startswith('.')])
for file_name in files:
    file_path = os.path.join('/Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/folded_merged', file_name)
    with open(file_path, 'r') as file:
        lines = [
            line for line in file
            if line.startswith("ATOM") and not line[12:16].strip().startswith("H")
        ]
    
    with open(file_path, 'w') as file:
        file.writelines(lines)

## Save files to ProteinCT/input_files/pdb

Only run this piece of code when you want to export PDB files to protein_CT/input_files/pdb/

In [39]:
#If folder does not exist, create folder
if not os.path.isdir('/Users/murielhammond/RP1/CT_code/Protein_CT/input_files/pdb'): os.makedirs('/Users/murielhammond/RP1/CT_code/Protein_CT/input_files/pdb')

#Copy pdb files to /Protein_CT/input_files/pdb
for num,files in enumerate(os.listdir('/Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged')):
    if files.endswith(('cif','pdb')):
        trajectory_PDB_files_preparation_path = os.path.join('/Users/murielhammond/RP1/CT_code/PDB_files_preparation/output/clean_pdb/disordered_merged', files)
        trajectory_protein_CT_path = '/Users/murielhammond/RP1/CT_code/Protein_CT/input_files/pdb'
        shutil.copy(trajectory_PDB_files_preparation_path, trajectory_protein_CT_path)